In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [12]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 데이터 로드 및 전처리

In [13]:
#데이터 경로
train_dir = '/data/face1/train'
test_dir = '/data/face1/test'

In [14]:
# ImageDataGenerator를 사용하여 데이터 로드 및 전처리
train_datagen = ImageDataGenerator(rescale=1./255)  # 이미지 정규화
test_datagen = ImageDataGenerator(rescale=1./255)   # 이미지 정규화

In [15]:
# 학습 데이터 생성기
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(48, 48),  # FER-2013 이미지 크기
    batch_size=64,
    color_mode='grayscale',  # 이미지가 흑백이므로 grayscale 사용
    class_mode='categorical'  # 레이블을 One-hot 인코딩
)

Found 28709 images belonging to 7 classes.


In [16]:
# 테스트 데이터 생성기
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(48, 48),
    batch_size=64,
    color_mode='grayscale',
    class_mode='categorical'
)

Found 7178 images belonging to 7 classes.


In [17]:
# 클래스 확인
print("클래스 이름:", train_generator.class_indices)

클래스 이름: {'angry': 0, 'disgust': 1, 'fear': 2, 'happy': 3, 'neutral': 4, 'sad': 5, 'surprise': 6}


# MobileNetV3 모델 정의

In [18]:
from tensorflow.keras.applications import MobileNetV3Small
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input

In [19]:
# MobileNetV3 모델 정의
input_shape = (48, 48, 1)  # FER-2013 이미지 크기
base_model = MobileNetV3Small(input_shape=input_shape, include_top=False, weights=None)

In [20]:
# MobileNetV3에 커스텀 레이어 추가
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Global Average Pooling
x = Dense(128, activation='relu')(x)  # Fully Connected Layer
output = Dense(7, activation='softmax')(x)  # 7개의 감정 클래스

In [21]:
# 최종 모델 정의
model = Model(inputs=base_model.input, outputs=output)

In [22]:
# 모델 요약 출력
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 48, 48, 1) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling           │ (None, 48, 48, 1) │          0 │ input_layer[0][0] │
│ (Rescaling)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv (Conv2D)       │ (None, 24, 24,    │        144 │ rescaling[0][0]   │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv_bn             │ (None, 24, 24,    │         64 │ conv[0][0]        │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 24, 24,    │          0 │ conv_bn[0][0]     │
│ (Activation)        │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 25, 25,    │          0 │ activation[0][0]  │
│ (ZeroPadding2D)     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 12, 12,    │        144 │ expanded_conv_de… │
│ (DepthwiseConv2D)   │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 12, 12,    │         64 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu (ReLU)        │ (None, 12, 12,    │          0 │ expanded_conv_de… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_sque… │ (None, 1, 1, 16)  │          0 │ re_lu[0][0]       │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_sque… │ (None, 1, 1, 8)   │        136 │ expanded_conv_sq… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_sque… │ (None, 1, 1, 8)   │          0 │ expanded_conv_sq… │
│ (ReLU)              │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_sque… │ (None, 1, 1, 16)  │        144 │ expanded_conv_sq… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 1, 1, 16)  │          0 │ expanded_conv_sq… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_1 (ReLU)      │ (None, 1, 1, 16)  │          0 │ add[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multiply (Multiply) │ (None, 1, 1, 16)  │          0 │ re_lu_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_sque… │ (None, 12, 12,    │          0 │ re_lu[0][0],      │
│ (Multiply)          │ 16)               │            │ multiply[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 12, 12,    │        256 │ expanded_conv_sq

 Total params: 1,013,591 (3.87 MB)

 Trainable params: 1,001,479 (3.82 MB)

 Non-trainable params: 12,112 (47.31 KB)

# 모델 컴파일

In [23]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# 모델 학습

In [24]:
history = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=20
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
449/449 ━━━━━━━━━━━━━━━━━━━━ 241s 505ms/step - accuracy: 0.3037 - loss: 1.7383 - val_accuracy: 0.2471 - val_loss: 1.9001
Epoch 2/20
449/449 ━━━━━━━━━━━━━━━━━━━━ 35s 79ms/step - accuracy: 0.3753 - loss: 1.5886 - val_accuracy: 0.2471 - val_loss: 1.8653
Epoch 3/20
449/449 ━━━━━━━━━━━━━━━━━━━━ 35s 79ms/step - accuracy: 0.4065 - loss: 1.5296 - val_accuracy: 0.2471 - val_loss: 1.8449
Epoch 4/20
449/449 ━━━━━━━━━━━━━━━━━━━━ 35s 79ms/step - accuracy: 0.4263 - loss: 1.4830 - val_accuracy: 0.2471 - val_loss: 1.8409
Epoch 5/20
449/449 ━━━━━━━━━━━━━━━━━━━━ 35s 79ms/step - accuracy: 0.4501 - loss: 1.4291 - val_accuracy: 0.1718 - val_loss: 1.8437
Epoch 6/20
449/449 ━━━━━━━━━━━━━━━━━━━━ 35s 79ms/step - accuracy: 0.4713 - loss: 1.3747 - val_accuracy: 0.2471 - val_loss: 1.8600
Epoch 7/20
449/449 ━━━━━━━━━━━━━━━━━━━━ 35s 79ms/step - accuracy: 0.4928 - loss: 1.3130 - val_accuracy: 0.2471 - val_loss: 1.8782
Epoch 8/20
449/449 ━━━━━━━━━━━━━━━━━━━━ 35s 79ms/step - accuracy: 0.5224 - loss: 1.2682 

In [26]:
loss, accuracy = model.evaluate(test_generator, verbose=2)
print(f"손실: {loss}, 정확도: {accuracy}")

113/113 - 2s - 21ms/step - accuracy: 0.1335 - loss: 45.9482
손실: 45.94823455810547, 정확도: 0.13346335291862488
